In [11]:
import unittest, time
def runtest(class_name):
    suite = unittest.TestSuite()
    suite = unittest.TestLoader().loadTestsFromTestCase(class_name)
    unittest.TextTestRunner().run(suite)

# 1.1 Is Unique

In [122]:
def unique(string):
    '''O(n)'''
    
    # Assuming character set is ASCII (128 characters), Unicode = 2 byte = 65536
    if len(string) > 128:
        return False

    char_set = [False for i in range(128)]
    for char in string:
        val = ord(char)
        if char_set[val]:
            return False
        char_set[val] = True
    return True

## if space complexity = O(1)

In [123]:
def unique2(string):
    '''O(n^2)'''
    
    l = len(string)
    for i in range(0, l-1):
        for j in range(i+1, l):
            if string[i] == string[j]:
                return False
    return True 

def unique3(string):
    '''O(nlogn)'''
    
    s = ''.join(sorted(string)) 
    for i in range(0, len(s)-1):
        if s[i] == s[i+1]:
            return False
    return True


In [222]:
class TestUnique(unittest.TestCase):
    def setUp(self):
        self.dataT = [('abcd'), ('s4fad'), ('')]
        self.dataF = [('23ds2'), ('hb 627jh=j ()')]
        self.test_funcs = [unique, unique2, unique3]

    def test_unique(self):
        # true check
        for unique in self.test_funcs:
            for test_string in self.dataT:
                actual = unique(test_string)
                self.assertTrue(actual)
            # false check
            for test_string in self.dataF:
                actual = unique(test_string)
                self.assertFalse(actual)
            
runtest(TestUnique)

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


# 1.2 Check Permutation

In [143]:
from collections import Counter
def check_permutation(s: str, t:str) -> bool:
    ''' use counter'''
    if len(s) != len(t):
        return False
    counter = Counter(s)
    for c in t:
        if counter[c] == 0:
            return False
        counter[c] -= 1
    return True

def check_permutation2(s: str, t:str) -> bool:
    ''' sorted and compare'''
    return sorted(s) == sorted(t)

In [149]:
class TestCheckPermutation(unittest.TestCase):
    def setUp(self):
        self.dataT = (
            ('abcd', 'bacd'),
            ('3563476', '7334566'),
            ('wef34f', 'wffe34'),
        )
        self.dataF = (
            ('abcd', 'd2cba'),
            ('2354', '1234'),
            ('dcw4f', 'dcw5f'),
        )

    def test_check_permutation(self):
        # true check
        for test_strings in self.dataT:
            result = check_permutation(*test_strings)
            self.assertTrue(result)
        # false check
        for test_strings in self.dataF:
            result = check_permutation(*test_strings)
            self.assertFalse(result)
    
    def test_check_permutation2(self):
        # true check
        for test_strings in self.dataT:
            result = check_permutation2(*test_strings)
            self.assertTrue(result)
        # false check
        for test_strings in self.dataF:
            result = check_permutation2(*test_strings)
            self.assertFalse(result)

runtest(TestCheckPermutation)

..
----------------------------------------------------------------------
Ran 2 tests in 0.005s

OK


# 1.3 URLify

In [174]:
def urlify(s: str, l: int) -> str:
    
    space = 0
    s = list(s)
    index = len(s)
    for i in range(l):
        if s[i] == ' ':
            space += 1
            
    for i in reversed(range(l)):
        ## Change from back to front
        if s[i] == ' ':
            s[index-3: index] = "%20"
            index -= 3
        else:
            s[index-1] = s[i]
            index -= 1
    return ''.join(s)

In [175]:
class TestUrlify(unittest.TestCase):
    '''Test Cases'''
    data = [
        ('much ado about nothing      ', 22, 'much%20ado%20about%20nothing'),
        ('Mr John Smith    ', 13, 'Mr%20John%20Smith')]

    def test_urlify(self):
        for [test_string, length, expected] in self.data:
            actual = urlify(test_string, length)
            self.assertEqual(actual, expected)
            
runtest(TestUrlify)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


# 1.4 Palindrome Permutation

In [32]:
## Aa~Zz -> 0~25, else -> -1
def char_number(c):
    a = ord('a')
    z = ord('z')
    A = ord('A')
    Z = ord('Z')
    val = ord(c)

    if a <= val <= z:
        return val - a
    elif A <= val <= Z:
        return val - A
    return -1

## hash table，O(N)
def palindrome_permutation(s: str) -> bool:
    s = s.replace(' ', '').lower()
    d = {}
    single = 0
    for c in s:
        if not d.get(c,False):
            d[c] = 1
        else:
            d[c] += 1
    for key, value in d.items():
        if value % 2 != 0:
            single += 1
    return single < 2

## 沿途檢查減少 hash table 迭代，o(N)
def palindrome_permutation2(s: str) -> bool:
    table = [0 for _ in range(ord('z') - ord('a') + 1)]
    countodd = 0
    for c in s:
        x = char_number(c)
        if x != -1:
            table[x] += 1
            if table[x] % 2:
                countodd += 1
            else:
                countodd -= 1

    return countodd <= 1

## 特殊做法位元向量，O(N)
def palindrome_permutation3(s: str) -> bool:
    bit_vector = 0
    for c in s:
        x = char_number(c)
        if x == -1:
            pass
        else:
            mask = 1 << x;
            ## 0 & 1 == 0 -> bit = 1
            if bit_vector & mask == 0:
                bit_vector |= mask
            ## 1 & 1 == 1 -> bit = 0
            else:
                bit_vector &= ~mask;
        
    ##  原本就是 0 或著 減 1 然後 AND = 0
    return bit_vector == 0 or (bit_vector & (bit_vector - 1)) ==0;

In [33]:
class TestPalindromePermutation(unittest.TestCase):
    def setUp(self):    
        self.data = [
        ('Tact Coa', True),
        ('jhsabckuj ahjsbckj', True),
        ('Able was I ere I saw Elba', True),
        ('So patient a nurse to nurse a patient so', False),
        ('Random Words', False),
        ('Not a Palindrome', False),
        ('no x in nixon', True),
        ('azAZ', True)]
        
        self.test_funcs = [palindrome_permutation, palindrome_permutation2, palindrome_permutation3]

    def test_palindrome_permutation(self):
        for palindrome_permutation in self.test_funcs:
            for [test_string, expected] in self.data:
                actual = palindrome_permutation(test_string)
                self.assertEqual(actual, expected)
runtest(TestPalindromePermutation)

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


# 1.5 One Way

In [224]:
def one_way(s1: str, s2: str) -> bool:
    edited = False
    l1, l2 = len(s1), len(s2)
    
    # Confirm that the difference is only one char
    if abs(l1 - l2) > 1:
        return False
    # let s1 < s2
    elif l1 > l2:
        s1, s2 = s2, s1


    # insert & delete
    if l1 != l2:
        i, j = 0, 0
        while i < len(s1) and j < len(s2):
            if s1[i] != s2[j]:        
                if edited:
                    return False
                edited = True
                j += 1
            else:
                i += 1
                j += 1
            
    # replace
    else:
        for c1, c2 in zip(s1, s2):
            if c1 != c2:
                if edited == True:
                    return False
                else:
                    edited = True

    return True      

In [225]:
class TestOneWay(unittest.TestCase):
    def setUp(self):
        self.test_cases = [
            # no changes
            ("pale", "pale", True),
            ("", "", True),
            # one insert
            ("pale", "ple", True),
            ("ple", "pale", True),
            ("pales", "pale", True),
            ("ples", "pales", True),
            ("pale", "pkle", True),
            ("paleabc", "pleabc", True),
            ("", "d", True),
            ("d", "de", True),
            # one replace
            ("pale", "bale", True),
            ("a", "b", True),
            ("pale", "ble", False),
            # multiple replace
            ("pale", "bake", False),
            # insert and replace
            ("pale", "pse", False),
            ("pale", "pas", False),
            ("pas", "pale", False),
            ("pkle", "pable", False),
            ("pal", "palks", False),
            ("palks", "pal", False),
            # permutation with insert shouldn't match
            ("ale", "elas", False),
        ]

    def test_one_away(self):  
        for [s1, s2, expected] in self.test_cases:
             self.assertEqual(one_way(s1, s2), expected)
            
runtest(TestOneWay)

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


# 1.6 Compress String

In [72]:
def string_compression(s: str) -> str:    
    letter = [s[0]]
    count = [0]
    ans = ''
    for c in s:
        if c == letter[-1]:
            count[-1] += 1
        else:
            letter.append(c)
            count.append(1)
            
    for l, c in zip(letter, count):
        ans += l + str(c)

    return ans if len(ans) < len(s) else s

In [73]:
class TestStringCompression(unittest.TestCase):
    def setUp(self):
        self.data = [
            ('aabcccccaaa', 'a2b1c5a3'),
            ('abcdef', 'abcdef')
        ]
        
    def test_string_compression(self):
        for [test_string, expected] in self.data:
            actual = string_compression(test_string)
            self.assertEqual(actual, expected)
            
runtest(TestStringCompression)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


# 1.7 Rotate Matrix

# 1.8 Zero Matrix

# 1.9 String Rotation